# Ann Arbor Parking Ticket Analysis
This analysis explores parking violation patterns in Ann Arbor using data obtained through Freedom of Information Act (FOIA) requests. The dataset spans from 2015 to early 2020, providing comprehensive insights into parking enforcement trends, vehicle distributions, and violation patterns across the city.

## Project Objectives
1. Extract and consolidate parking ticket data from multiple excel files
2. Examine the temporal distribution of different violation types
3. Study out-of-state vehicle patterns with a focus on New York registrations
4. Investigate Michigan license plate formats and their frequencies

# 1. Data Processing and Integration
## Overview

The initial phase involves combining multiple Excel files containing parking ticket records. Each file requires careful handling due to varying sheet structures and header formats.

In [3]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

def load_ticket_data():
    def process_file(file):
        data = pd.read_excel(file, sheet_name=None, header=None)
        frames = []

        for sheet_name, df in data.items():
            print(f"Processing sheet: {sheet_name} from file: {file}")  

            
            if not df.empty:
                if file.endswith('2020.xls') and sheet_name == 'Sheet1':  
                    df = df.iloc[2:-1]  
                elif sheet_name == 'Sheet1':  
                    df = df.iloc[2:]
                elif sheet_name in ['Sheet2', 'Sheet3']:  
                    df = df.iloc[1:] 
                    if sheet_name == 'Sheet3':
                        df = df.iloc[:-1]  

                if len(df.columns) >= 14:
                    df.columns = [
                        'Ticket #', 'Badge', 'Issue Date', 'IssueTime', 'Plate', 'State',
                        'Make', 'Model', 'Violation', 'Description', 'Location', 'Meter',
                        'Fine', 'Penalty',
                    ]
                    frames.append(df)
                else:
                    print(f"Skipping sheet: {sheet_name} due to unexpected column count in {file}")

        if frames:
            return pd.concat(frames, ignore_index=True)

        return pd.DataFrame() 

    files = [
        'AnnArbor-TicketViolation2015.xls',
        'AnnArbor-TicketViolation2016.xls',
        'AnnArbor-TicketViolation2017.xls',
        'AnnArbor-TicketViolation2018.xls',
        'AnnArbor-TicketViolation2019.xls',
        'AnnArbor-TicketViolation-jan2020.xls',
    ]

    all_df = []
    for file in files:
        df = process_file(file)
        if not df.empty:
            all_df.append(df)
        else:
            print(f"No valid data found in file: {file}")  

    if all_df:
        combined_df = pd.concat(all_df, ignore_index=True)
        print(f"Combined DataFrame shape: {combined_df.shape}") 
        return combined_df
    else:
        raise ValueError("No valid data found in any files.")

df = load_ticket_data()
print(f"DataFrame Shape: {df.shape}")
df.head()

Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2019.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2019.xls
Processing sheet: Sheet3 from file

,Ticket #,Badge,Issue Date,IssueTime,Plate,State,Make,Model,Violation,Description,Location,Meter,Fine,Penalty
0,H000210594,036,2015-01-01 00:00:00,2214,LAS5658,OH,SUBA,NaN,A04,NO PRKNG ANYTME,525 ELM,NaN,35,20
1,2100005782,821,2015-01-02 00:00:00,0824,DEZ4465,MI,FORD,NaN,A01,EXPIRED METER,600 BLK OF STATE SOU,4006A,10,0
2,2110008524,826,2015-01-02 00:00:00,1719,DCM1327,MI,SATU,NaN,A01,EXPIRED METER,FARMER'S MARKET,17,10,0
3,2110008525,826,2015-01-02 00:00:00,1725,BAX385,IA,CHEV,NaN,A01,EXPIRED METER,FARMER'S MARKET,35,10,0
4,2100005834,821,2015-01-02 00:00:00,1344,2LEH1,MI,FORD,NaN,A04,NO PRKNG ANYTME,600 BLK OF WILLIAM E,NaN,25,0


## Data Structure 
After processing, our dataset contains the following key information:

Ticket information (ID, date, time)
Vehicle details (plate, state, make, model)
Violation specifics (type, description, location)
Financial data (fine amount, penalties)

# 2. Temporal Analysis of Violations
## Methodology
We categorize violations across three distinct time periods to understand enforcement patterns:

Morning: 3:00 AM - 11:59 AM
Afternoon: 12:00 PM - 5:59 PM
Evening: 6:00 PM - 2:59 AM

In [4]:
def generate_descriptors(df):
    df['IssueTime'] = df['IssueTime'].astype(str).str.zfill(4)
    df['IssueTime'] = pd.to_numeric(df['IssueTime'], errors='coerce')
   
    df = df.dropna(subset=['Description'])
    
    def count_descriptions_in_period(df, start, end):
        if start < end:
            mask = (df['IssueTime'] >= start) & (df['IssueTime'] < end)
        else:  # For evening period crossing midnight
            mask = (df['IssueTime'] >= start) | (df['IssueTime'] < end)
        return df.loc[mask, 'Description'].value_counts()
    
    morning_counts = count_descriptions_in_period(df, 300, 1200)
    afternoon_counts = count_descriptions_in_period(df, 1200, 1800)
    evening_counts = count_descriptions_in_period(df, 1800, 300)
    
    all_descriptions = list(morning_counts.index.union(afternoon_counts.index).union(evening_counts.index))
    result = pd.DataFrame(index=['Morning', 'Afternoon', 'Evening'], columns=all_descriptions).fillna(0)
    
    result.loc['Morning', morning_counts.index] = morning_counts
    result.loc['Afternoon', afternoon_counts.index] = afternoon_counts
    result.loc['Evening', evening_counts.index] = evening_counts
    
    return result.fillna(0)

descriptors = generate_descriptors(df)
print(descriptors)

           12 INCHES CURB  15' FIRE HYDRAN  ABANDONED VEHIC  ACROSS LINE  \
Morning               174             1152              159         3049   
Afternoon             345             2183              292         1752   
Evening                36              613                5          282   

           ACROSS MARKED SPACES  AHEAD SET/BACK  BACKED IN STALL  \
Morning                       0             465              483   
Afternoon                     1             692             1078   
Evening                       0              56                0   

           BLOCK CROSSWALK  BLOCKING ALLEY  BLOCKING CRSSWA  ...  \
Morning                  6             113               61  ...   
Afternoon               18             208              113  ...   
Evening                  2              13               17  ...   

           PARKING ON WALK  PKD IN INTERSECTION  PRIVATE PARK  \
Morning               1318                    0            69   
Afternoon          

# 3. Vehicle Origin Analysis
## New York Vehicle Profile
We specifically analyze vehicles with New York registration to understand out-of-state parking patterns.

In [5]:
def common_car_make():
    df = load_ticket_data()
    
    ny_tickets = df[df['State'] == 'NY']
    
    if ny_tickets.empty:
        return "No NY plates found"

    most_common_make = ny_tickets['Make'].value_counts().idxmax()
    
    return most_common_make

answer = common_car_make()

print(answer)

Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2019.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2019.xls
Processing sheet: Sheet3 from file

# 4. Michigan License Plate Analysis
## Pattern Recognition 
We examine different Michigan license plate formats:

-Standard format (ABC1234)

-Legacy format (ABC123)

-Numerical prefix format (123ABC)

-Custom/Vanity plates

In [6]:
import re

def fine_per_plates(df=None):
    if df is None:
        df = load_ticket_data() 

    mi_tickets = df[df['State'] == 'MI']
    print(f"Total Michigan tickets: {len(mi_tickets)}") 

    plates = mi_tickets['Plate'].dropna().astype(str).str.strip().str.upper()


    pattern_ABC1234 = r'^[A-Z]{3}[0-9]{4}$'  
    pattern_ABC123 = r'^[A-Z]{3}[0-9]{3}$'   
    pattern_123ABC = r'^[0-9]{3}[A-Z]{3}$'   

    
    count_ABC1234 = plates.str.match(pattern_ABC1234).sum()
    count_ABC123 = plates.str.match(pattern_ABC123).sum()
    count_123ABC = plates.str.match(pattern_123ABC).sum()

    total_plates = len(plates)
    count_vanity = total_plates - (count_ABC1234 + count_ABC123 + count_123ABC)
  
    count_vanity += len(mi_tickets) - len(plates)
 
    print(f"Count ABC1234: {count_ABC1234}")
    print(f"Count ABC123: {count_ABC123}")
    print(f"Count 123ABC: {count_123ABC}")
    print(f"Count Vanity: {count_vanity}")
   
    return {
        "ABC1234": count_ABC1234,
        "ABC123": count_ABC123,
        "123ABC": count_123ABC,
        "vanity": count_vanity
    }

result = fine_per_plates()
print(result)

Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2015.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2016.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2017.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet3 from file: AnnArbor-TicketViolation2018.xls
Processing sheet: Sheet1 from file: AnnArbor-TicketViolation2019.xls
Processing sheet: Sheet2 from file: AnnArbor-TicketViolation2019.xls
Processing sheet: Sheet3 from file